<img src="https://static.wixstatic.com/media/a9ca5e_825bd4d39e7d468faf735b801fa3dea4~mv2.png/v1/fill/w_1458,h_246,al_c,usm_0.66_1.00_0.01/a9ca5e_825bd4d39e7d468faf735b801fa3dea4~mv2.png" width="200">

# Twinfield api
 __Notebook voor het uitvragen van twinfield dataset__
 
 __Auteur:__ Melvin Folkers (Yellow Stacks B.V.)<br>
 __Versie:__ 29 januari 2018<br>
 __Doel:__ 

<h1>Overzicht<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#parameters" data-toc-modified-id="parameters-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>parameters</a></span><ul class="toc-item"><li><span><a href="#run-parameters" data-toc-modified-id="run-parameters-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>run parameters</a></span></li><li><span><a href="#import-libraries" data-toc-modified-id="import-libraries-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>import libraries</a></span></li><li><span><a href="#juno-settings" data-toc-modified-id="juno-settings-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>juno settings</a></span></li></ul></li><li><span><a href="#functions-and-classes" data-toc-modified-id="functions-and-classes-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>functions and classes</a></span><ul class="toc-item"><li><span><a href="#General-functions" data-toc-modified-id="General-functions-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>General functions</a></span></li><li><span><a href="#session-id" data-toc-modified-id="session-id-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>session id</a></span></li><li><span><a href="#Officecodes" data-toc-modified-id="Officecodes-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Officecodes</a></span></li><li><span><a href="#metadata" data-toc-modified-id="metadata-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>metadata</a></span></li><li><span><a href="#data" data-toc-modified-id="data-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>data</a></span></li><li><span><a href="#select-office" data-toc-modified-id="select-office-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>select office</a></span></li></ul></li><li><span><a href="#session-ID-Twinfield" data-toc-modified-id="session-ID-Twinfield-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>session ID Twinfield</a></span></li><li><span><a href="#Determine-offices" data-toc-modified-id="Determine-offices-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Determine offices</a></span><ul class="toc-item"><li><span><a href="#select-office" data-toc-modified-id="select-office-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>select office</a></span></li></ul></li><li><span><a href="#Browse-data" data-toc-modified-id="Browse-data-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Browse data</a></span><ul class="toc-item"><li><span><a href="#Metadata" data-toc-modified-id="Metadata-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Metadata</a></span></li><li><span><a href="#Data" data-toc-modified-id="Data-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Data</a></span></li></ul></li></ul></div>

## parameters

### run parameters

In [1]:
user = 'Python'
password = 'U3RhZmZpbmcyMDE5IQ==\n'
organisation = 'Associates'

### import libraries

In [2]:
import os 
import pandas as pd 
import math
from datetime import date, timedelta
import numpy as np
import codecs
import requests

In [3]:
import xml.etree.ElementTree as ET

### juno settings

In [4]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>")) # maakt de jupyter notebook cellen 100% breed
from IPython.display import IFrame
%matplotlib inline 
# voor het plotten van grafieken

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" # om meerdere grafieken/tabellen in 1 cel te kunnen laten zien.

pd.options.display.max_columns = None # laat alle kolommen zien
pd.options.display.float_format = '{:,.2f}'.format

from IPython.display import Image
from IPython.core.display import HTML 



## functions and classes

### General functions

In [5]:
def str_decoding(base64_str):
    
    decoded = codecs.decode(base64_str.encode(), 'base64').decode()

    return decoded

def str_encoding(str_input):
    encoded = codecs.encode(str_input.encode(), 'base64')

    return encoded

### session id

In [6]:
class SessionParameters():

    def __init__(self,user, pw, organisation):
       
        self.user = user
        self.password = pw 
        self.organisation = organisation
        
        self.url =  'https://login.twinfield.com/webservices/session.asmx'
        self.header = {'Content-Type':'text/xml',
        'Accept-Charset':'utf-8'}
        
        self.ns = {'env': 'http://schemas.xmlsoap.org/soap/envelope/',
                   'tw':'http://www.twinfield.com/'}
        
        self.body = f'''<?xml version="1.0" encoding="utf-8"?>
        <soap:Envelope xmlns:soap="http://schemas.xmlsoap.org/soap/envelope/" xmlns:xsi="http://www.w3.org/2001/XmlSchema-instance" xmlns:xsd="http://www.w3.org/2001/XmlSchema">
          <soap:Body>
            <Logon xmlns="http://www.twinfield.com/">
              <user>{user}</user>
              <password>{str_decoding(password)}</password>
              <organisation>{organisation}</organisation>
            </Logon>
          </soap:Body>
        </soap:Envelope>'''
    
    
    def parse_session_id(self, root):
    
        header = root.find('env:Header/tw:Header', self.ns)
        session_id = header.find('tw:SessionID', self.ns).text
    
        return session_id
    

    def get_session_id(self, url, header, body):
    
        response = requests.post(url = url, headers = header , data = body)

        if response:
            session_id = SessionParameters.parse_session_id(self, root = ET.fromstring(response.text)) # lees de response uit
        else:
            print('niet gelukt om data binnen te halen')
            session_id = response
            
            self.session_id = session_id

        return session_id



### Officecodes

In [7]:
class GetOfficeCodes():
    
    def __init__(self, session_id):

        self.url = 'https://c4.twinfield.com/webservices/processxml.asmx?wsdl'

        self.header = {'Content-Type':'text/xml',
                'Accept-Charset':'utf-8'}

        self.body = f'''<?xml version="1.0" encoding="utf-8"?>
        <soapenv:Envelope xmlns:soapenv="http://schemas.xmlsoap.org/soap/envelope/" xmlns:twin="http://www.twinfield.com/">
        <soapenv:Header>
            <twin:Header>
                 <twin:SessionID>{session_id}</twin:SessionID> 
              </twin:Header>
           </soapenv:Header>
           <soapenv:Body>
              <twin:ProcessXmlString>
                 <twin:xmlRequest><![CDATA[<list><type>offices</type></list>]]></twin:xmlRequest>
              </twin:ProcessXmlString>
           </soapenv:Body>
        </soapenv:Envelope>'''

        self.ns = {'env': 'http://schemas.xmlsoap.org/soap/envelope/',
                       'tw':'http://www.twinfield.com/'}

    def get_offices(self, url, header, body):

        response = requests.post(url = url, headers = header , data = body)
        
        if response:
            offices = GetOfficeCodes.parse_session_response(self, response)
        else:
            print('niet gelukt om data binnen te halen')
            offices = pd.DataFrame()
            
        return offices

    def parse_session_response(self, response):

        root = ET.fromstring(response.text)

        body = root.find('env:Body', self.ns)

        data = body.find('tw:ProcessXmlStringResponse/tw:ProcessXmlStringResult', self.ns)

        data = ET.fromstring(data.text)

        df_ttl = pd.DataFrame()

        for child in data:

            df = pd.DataFrame(data = child.attrib, index = [child.text])
            df_ttl = pd.concat([df_ttl , df], axis = 0)

        return df_ttl


### metadata

In [8]:
def get_metadata(office,session_id, code):
    
    xml = f'''<read>
    <type>browse</type>
    <office>{office}</office>
    <code>{code}</code>
    </read>'''
    
    body = f'''<?xml version="1.0" encoding="utf-8"?>
        <soapenv:Envelope xmlns:soapenv="http://schemas.xmlsoap.org/soap/envelope/" xmlns:twin="http://www.twinfield.com/">
        <soapenv:Header>
            <twin:Header>
            <twin:SessionID>{session_id}</twin:SessionID> 
              </twin:Header>
           </soapenv:Header>
           <soapenv:Body>
              <twin:ProcessXmlString>
                 <twin:xmlRequest><![CDATA[{xml}]]></twin:xmlRequest>
              </twin:ProcessXmlString>
           </soapenv:Body>
        </soapenv:Envelope>'''
    
    
    response = requests.post(url = auth.url, headers = auth.header , data = body)

    root = ET.fromstring(response.text)

    body = root.find('env:Body', auth.ns)

    data = body.find('tw:ProcessXmlStringResponse/tw:ProcessXmlStringResult', auth.ns)

    data = ET.fromstring(data.text)

    
    metadata = parse_metadata_response(data)
    
    return metadata


def parse_metadata_response(data):
    
    
    col = data.find('columns')
    
    rec = list()

    for records in col:
      
        ttl = dict()
        for record in records:
            ttl[record.tag] = record.text

        rec.append(ttl)

    df = pd.DataFrame(rec)
    
    return df
    

### data

### select office

In [9]:
def select_office(office,session_id):
    
    
    print('selecting office: {office}...')
    
    url = 'https://c4.twinfield.com/webservices/session.asmx?wsdl'

    body = f'''<?xml version="1.0" encoding="utf-8"?>
    <soap:Envelope xmlns:soap="http://schemas.xmlsoap.org/soap/envelope/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema">
        <soap:Header>
            <Header xmlns="http://www.twinfield.com/">
                <SessionID>{session_id}</SessionID>
            </Header>
        </soap:Header>
        <soap:Body>
            <SelectCompany xmlns="http://www.twinfield.com/">
                <company>{office}</company>
            </SelectCompany>
        </soap:Body>
    </soap:Envelope>'''

    response = requests.post(url = url, headers = auth.header , data = body)

    root = ET.fromstring(response.text)

    body = root.find('env:Body', auth.ns)

    data = body.find('tw:SelectCompanyResponse/tw:SelectCompanyResult', auth.ns)

    pass_fail = data.text
    
    print(pass_fail)




## session ID Twinfield

In [10]:
login = SessionParameters(user, password, organisation)

In [11]:
session_id = login.get_session_id(login.url, login.header, login.body)

In [12]:
session_id

'c635ba38-587e-413f-9de4-18f6de48a75f'

In [13]:
print(f'session id: {session_id}')

session id: c635ba38-587e-413f-9de4-18f6de48a75f


## Determine offices

In [14]:
auth = GetOfficeCodes(session_id)

In [15]:
offices = auth.get_offices(auth.url, auth.header, auth.body)

In [16]:
offices.shape

(61, 2)

### select office

In [46]:
office = '1060266'
code = '030_1'

In [37]:
select_office(office, session_id)

selecting office: {office}...
Ok


## Browse data

### Metadata 

In [38]:
metadata = get_metadata(office = office, code = code, session_id= session_id)

In [39]:
#metadata = metadata[(['field', 'label'])]

In [40]:
metadata.set_index('field', inplace = True)

fieldmapping = metadata[ 'label'].to_dict()

In [45]:
metadata

,ask,finderparam,from,label,operator,to,visible
field,,,,,,,
fin.trs.head.office,false,None,None,Administratie,none,None,true
fin.trs.head.officename,false,None,None,Adm.naam,none,None,true
fin.trs.head.yearperiod,true,None,None,Jaar/periode (JJJJ/PP),between,None,false
fin.trs.head.status,true,None,normal,Status,equal,None,true
fin.trs.line.dim1,true,None,None,Grootboekrek.,between,None,true
fin.trs.line.dim1name,false,None,None,Grootboekrek.naam,none,None,true
fin.trs.line.dim1type,false,None,None,Dimensietype 1,none,None,true
fin.trs.line.valuesigned,false,None,None,Bedrag,none,None,true
fin.trs.line.basevaluesigned,false,None,None,Basisbedrag,none,None,true


### Data

In [41]:
def get_data(office,session_id, code):
    
    xml = f'''<read>
    <type>browse</type>
    <office>{office}</office>
    <code>{code}</code>
    </read>'''
    
    body = f'''<?xml version="1.0" encoding="utf-8"?>
        <soapenv:Envelope xmlns:soapenv="http://schemas.xmlsoap.org/soap/envelope/" xmlns:twin="http://www.twinfield.com/">
        <soapenv:Header>
            <twin:Header>
            <twin:SessionID>{session_id}</twin:SessionID> 
              </twin:Header>
           </soapenv:Header>
           <soapenv:Body>
              <twin:ProcessXmlString>
                 <twin:xmlRequest><![CDATA[{xml}]]></twin:xmlRequest>
              </twin:ProcessXmlString>
           </soapenv:Body>
        </soapenv:Envelope>'''
    
    response = requests.post(url = auth.url, headers = auth.header , data = body)

    root = ET.fromstring(response.text)

    body = root.find('env:Body', auth.ns)

    data = body.find('tw:ProcessXmlStringResponse/tw:ProcessXmlStringResult', auth.ns)

    #data = ET.fromstring(data.text)
    
    return data


def parse_data_response(data):
    
    
    col = data.find('columns')
    
    rec = list()

    for records in col:
      
        ttl = dict()
        for record in records:
            ttl[record.tag] = record.text

        rec.append(ttl)

    df = pd.DataFrame(rec)
    
    return df
    

In [42]:
#data = get_data(office, session_id, code)

# Development

In [43]:
code = '030_1'
period = '2019'

In [44]:
body = f'''<?xml version="1.0" encoding="utf-16"?>
<soap:Envelope xmlns:soap="http://schemas.xmlsoap.org/soap/envelope/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema">
  <soap:Header>
	 <Header xmlns="http://www.twinfield.com/">
	  <SessionID>{session_id}</SessionID>
	</Header>
  </soap:Header>
  <soap:Body>
	 <ProcessXmlDocument xmlns="http://www.twinfield.com/">
		<xmlRequest>
<columns code="{code}">
<column xmlns="">
      <field>fin.trs.head.office</field>
   </column>
<column xmlns="">
      <field>fin.trs.head.officename</field>
      <label>Adm.naam</label>
      <visible>true</visible>
   </column> 
   <column xmlns="">
      <field>fin.trs.head.yearperiod</field>
      <label>jaar/periode</label>
      <visible>false</visible>
      <ask>true</ask>
      <operator>between</operator>
      <from>{period}/00</from>
      <to>{period}/55</to>
   </column> 
<column xmlns="">
      <field>fin.trs.head.year</field>
      <label>Jaar</label>
      <visible>true</visible>
   </column>
<column xmlns="">
      <field>fin.trs.head.period</field>
      <label>Periode</label>
      <visible>true</visible>
   </column>
<column xmlns="">
      <field>fin.trs.head.code</field>
      <label>Dagboek</label>
      <visible>true</visible>
   </column>
<column xmlns="">
      <field>fin.trs.head.number</field>
      <label>Boekingsnummer</label>
      <visible>true</visible>
   </column>
<column xmlns="">
      <field>fin.trs.head.status</field>
      <label>Status</label>
      <visible>true</visible>
   </column>
<column xmlns="">
      <field>fin.trs.head.date</field>
      <label>Boekdatum</label>
      <visible>true</visible>
   </column>
<column xmlns="">
      <field>fin.trs.head.curcode</field>
      <label>Valuta</label>
      <visible>true</visible>
   </column>
<column xmlns="">
      <field>fin.trs.head.relationname</field>
      <label>Relatienaam</label>
      <visible>true</visible>
   </column>
<column xmlns="">
      <field>fin.trs.head.inpdate</field>
      <label>Invoerdatum</label>
      <visible>true</visible>
   </column>
<column xmlns="">
      <field>fin.trs.head.username</field>
      <label>Gebruikersnaam</label>
      <visible>true</visible>
   </column>
<column xmlns="">
      <field>fin.trs.line.dim1</field>
      <label>Grootboekrek.</label>
      <visible>true</visible>
   </column>
<column xmlns="">
      <field>fin.trs.line.dim1name</field>
      <label>Grootboekrek.naam</label>
      <visible>true</visible>
   </column>
<column xmlns="">
      <field>fin.trs.line.dim1type</field>
      <label>Dimensietype 1</label>
      <visible>true</visible>
   </column>
<column xmlns="">
      <field>fin.trs.line.dim2</field>
      <label>Kpl./rel.</label>
      <visible>true</visible>
   </column>
<column xmlns="">
      <field>fin.trs.line.dim2name</field>
      <label>Kpl.-/rel.naam</label>
      <visible>true</visible>
   </column>
<column xmlns="">
      <field>fin.trs.line.dim2type</field>
      <label>Dimensietype 2</label>
      <visible>true</visible>
   </column>
<column xmlns="">
      <field>fin.trs.line.dim3</field>
      <label>Act./proj.</label>
      <visible>true</visible>
   </column>
<column xmlns="">
      <field>fin.trs.line.dim3name</field>
      <label>Act.-/proj.naam</label>
      <visible>true</visible>
   </column>
<column xmlns="">
      <field>fin.trs.line.dim3type</field>
      <label>Dimensietype 3</label>
      <visible>true</visible>
   </column>
<column xmlns="">
      <field>fin.trs.line.valuesigned</field>
      <label>Bedrag</label>
      <visible>true</visible>
   </column>
<column xmlns="">
      <field>fin.trs.line.basevaluesigned</field>
      <label>Basisbedrag</label>
      <visible>true</visible>
   </column>
<column xmlns="">
      <field>fin.trs.line.repvaluesigned</field>
      <label>Rapportagebedrag</label>
      <visible>true</visible>
   </column>
<column xmlns="">
      <field>fin.trs.line.debitcredit</field>
      <label>D/C</label>
      <visible>true</visible>
   </column>
<column xmlns="">
      <field>fin.trs.line.vatcode</field>
      <label>Btw-code</label>
      <visible>true</visible>
   </column>
<column xmlns="">
      <field>fin.trs.line.vatbasevaluesigned</field>
      <label>Btw-bedrag</label>
      <visible>true</visible>
   </column>
<column xmlns="">
      <field>fin.trs.line.quantity</field>
      <label>Aantal</label>
      <visible>true</visible>
   </column>
<column xmlns="">
      <field>fin.trs.line.chequenumber</field>
      <label>Cheque</label>
      <visible>true</visible>
   </column>
<column xmlns="">
      <field>fin.trs.line.description</field>
      <label>Omschrijving</label>
      <visible>true</visible>
   </column>
<column xmlns="">
      <field>fin.trs.line.invnumber</field>
      <label>Factuurnummer</label>
      <visible>true</visible>
   </column>
<column xmlns="">
      <field>fin.trs.line.dim1group1</field>
      <label>Groep 1</label>
      <visible>true</visible>
   </column>
<column xmlns="">
      <field>fin.trs.line.dim1group1name</field>
      <label>Groepnaam 1</label>
      <visible>true</visible>
   </column>
<column xmlns="">
      <field>fin.trs.line.dim1group2</field>
      <label>Groep 2</label>
      <visible>true</visible>
   </column>
<column xmlns="">
      <field>fin.trs.line.dim1group2name</field>
      <label>Groepnaam 2</label>
      <visible>true</visible>
   </column>
<column xmlns="">
      <field>fin.trs.line.dim1group3</field>
      <label>Groep 3</label>
      <visible>true</visible>
   </column>
<column xmlns="">
      <field>fin.trs.line.dim1group3name</field>
      <label>Groepnaam 3</label>
      <visible>true</visible>
   </column>
<column xmlns="">
      <field>fin.trs.line.freetext1</field>
      <label>Vrij tekstveld 1</label>
      <visible>true</visible>
   </column>
<column xmlns="">
      <field>fin.trs.line.freetext2</field>
      <label>Vrij tekstveld 2</label>
      <visible>true</visible>
   </column>
<column xmlns="">
      <field>fin.trs.line.freetext3</field>
      <label>Vrij tekstveld 3</label>
      <visible>true</visible>
   </column>
<column xmlns="">
      <field>fin.trs.head.origin</field>
      <label>Boekingsoorsprong</label>
      <visible>true</visible>
   </column>
<column xmlns="">
      <field>fin.trs.mng.type</field>
      <label>transactie type groep</label>
      <visible>true</visible>
   </column>
</columns>
		</xmlRequest>
	</ProcessXmlDocument>
  </soap:Body>
</soap:Envelope>'''

In [26]:
response = requests.post(url = auth.url, headers = auth.header , data = body)
root = ET.fromstring(response.text)
body = root.find('env:Body', auth.ns)
data = body.find('tw:ProcessXmlDocumentResponse/tw:ProcessXmlDocumentResult', auth.ns)

In [27]:
test = data.find('browse')
tr = test.findall('tr')

In [28]:
ttl_records = list() 

for trans in tr:
    
    info = dict()
    
    for col in trans:
        
        val = col.text
        
        if 'field' in  col.attrib.keys():
            name = col.attrib['field']

            info[name] = val
    
    ttl_records.append(info)

In [29]:
data = pd.DataFrame(ttl_records)

In [30]:
data.shape

(20804, 41)

In [31]:
data.rename(fieldmapping, axis = 1, inplace = True)

In [32]:
#data.dropna(axis = 1, inplace = True)